### Imports

In [79]:
import numpy as np
import bond_pricing

### Day Count Convention

The relevant time period in most financial markets is based on the number of days between the starting and ending dates. In other words, “parking lot rules”(whereby both the starting and ending dates count) do not apply.

In [71]:
from datetime import date
from pandas import to_datetime, Timedelta, DateOffset
from importlib import import_module

daycount_dict = {'actual/360': 'actual360', 
                 'actual/365': 'actual365',
                 'actual/actual': 'actualactual', 
                 'thirty/360': 'thirty360'}

settle = to_datetime('2024-1-12')
mat = to_datetime('2024-03-12')
convention = 'actual/365'

In [83]:
def days_year(settle, mat, convention='actual/360'):
    name = 'isda_daycounters.'+ daycount_dict[convention]
    days = import_module(name).day_count(settle, mat)
    if convention == "actual/360" or convention == "thirty/360":
      year = 360
    elif convention == "actual/365":
      year = 365
    else:
      oneyear = settle + DateOffset(years=1)
      year = (oneyear - settle).days
    return days, year
    
days, year = days_year(settle, mat, convention=convention)
print(f'{days = }\n{year = }')

days = 59
year = 365


In [73]:
def year_frac(settle, mat, convention='actual/360', inverse=False):
    name = 'isda_daycounters.'+ daycount_dict[convention]
    fraction = import_module(name).year_fraction(settle, mat)
    if inverse:
      return fraction
    else:
      return 1/fraction

dayYear = year_frac(settle, mat, convention=convention, inverse=False)
print(f'{dayYear = }')
print(year/days)

yearDay = year_frac(settle, mat, convention=convention, inverse=True)
print(f'{yearDay = }')
print(days/year)

dayYear = 6.083333333333334
6.083333333333333
yearDay = 0.1643835616438356
0.1643835616438356


### Add-On Rate

The following rates are generally quoted on an add-on rate basis:
- Commercial bank loans and deposits
- Certificates of deposit (CD)
- Repos
- Fed funds
- LIBOR
- SOFR
- Commercial paper (CP) in Euromarkets

Add-on rates follow a simple interest calculations. The **interest is added on to the principal amount** to get the **redemption payment** at maturity.

$$FV=PV+\left[PV \times AOR \times \frac{Days}{Year}\right]$$
$$FV=PV\times\left[1 + \left(AOR \times \frac{Days}{Year}\right)\right]$$
$$\frac{FV}{PV}=\left[1 + \left(AOR \times \frac{Days}{Year}\right)\right]$$
$$PV=\frac{FV}{\left[1 + \left(AOR \times \frac{Days}{Year}\right)\right]}$$
$$AOR=\left(\frac{Year}{Days}\right)\times\left(\frac{FV-PV}{PV}\right)$$
$$AOR\times\left(\frac{Days}{Year}\right)=\left(\frac{FV-PV}{PV}\right)$$

- AOR is the quoted add-on rate (annual percentage rate (APR))
- PV the present value (the initial principal amount)
- FV the future value (the redemption payment including interest)
- Days the number of days until maturity, and 
- Year the number of days in the year

In [17]:
def addon_fv(pv, aor, days, year=360):
    return pv * (1 + (aor * days  / year))

pv = 1_000_000
aor = 0.039
days = 180
year = 360

fv = addon_fv(pv, aor, days, year)
print(f'fv ={round(fv, 2): ,}')
periodicity = year/days
print(f'{periodicity = }')

fv = 1,019,500.0
periodicity = 2.0


In [18]:
def addon_pv(fv, aor, days, year=360):
    return fv / (1 + (aor * days  / year))

fv = 1_019_500
aor = 0.0372
days = 120
year = 360

pv = addon_pv(fv, aor, days, year)
print(f'pv ={round(pv, 2): ,}')
periodicity = year/days
print(f'{periodicity = }')

pv = 1,007,013.04
periodicity = 3.0


In [19]:
def addon_rate(pv, fv, days, year=360):
    return (year/days) * (fv - pv)/pv

pv = 1_000_000
fv = 1_007_013.04
days = 60
year = 360

aor = addon_rate(pv, fv, days, year)
print(f'{aor = : 0.4f}')
periodicity = year/days
print(f'{periodicity = }')

aor =  0.0421
periodicity = 6.0


In [12]:
def periodicity(days, year):
    return year / days

days = 90
year = 360

print(periodicity(days, year))

4.0


### Discount Rate

The following rates are generally quoted on a discount rate basis:
- T-Bills
- Commercial paper (CP) 
- Banker's acceptance

The price of the security is a **discount from the face value**. The future (or face) value times the annual discount rate times the fraction of the year. Interest is not “added on” to the principal; instead it is included in the face value.

$$PV = FV - \left[FV \times DR \times \frac{Days}{Year}\right]$$
$$PV = FV \times \left[1 - \left(DR \times \frac{Days}{Year}\right)\right]$$
$$\frac{PV}{FV} = \left[1 - \left(DR \times \frac{Days}{Year}\right)\right]$$
$$FV = \frac{PV}{\left[1 - \left(DR \times \frac{Days}{Year}\right)\right]}$$
$$DR=\left(\frac{Year}{Days}\right)\times\left(\frac{FV-PV}{FV}\right)$$
$$DR\times\left(\frac{Days}{Year}\right)=\left(\frac{FV-PV}{FV}\right)$$

- DR is the discount rate
- PV the present value (the initial principal amount)
- FV the future value (the redemption payment including interest)
- Days the number of days until maturity, and 
- Year the number of days in the year

The “amount” of a transaction is the face value (the FV) for instruments quoted on a discount rate basis. In contrast, the “amount” is the original principal (the PV at issuance) for money market securities quoted on an add-on rate basis.

Note:
$$PV = FV - \left[FV \times DR \times \frac{Days}{Year}\right]$$
Can be simplified to:
$$PV = FV - \text{Dollar Discount}$$
where $\text{Dollar Discount} = FV  \times DR \times \frac{Days}{Year}$

In [82]:
def dr_pv(fv, dr, days, year=360):
    return fv * (1 - (dr * days  / year))

fv = 1_000_000
dr = 0.038
days = 180
year = 360

pv = dr_pv(fv, dr, days, year)
print(f'pv ={round(pv, 2): ,}')
periodicity = year/days
print(f'{periodicity = }')

pv = 981,000.0
periodicity = 2.0


In [40]:
def dr_pv(fv, dr, days, year=360):
    return fv * (1 - (dr * days  / year))

fv = 1_000_000
dr = 0.0335
days = 30
year = 360

pv = dr_pv(fv, dr, days, year)
print(f'pv ={round(pv, 2): ,}')
periodicity = year/days
print(f'{periodicity = }')

pv = 997,208.33
periodicity = 12.0


In [41]:
def dr_fv(pv, dr, days, year=360):
    return pv / (1 - (dr * days  / year))

pv = 1_019_500
dr = 0.0372
days = 120
year = 360

fv = dr_fv(pv, dr, days, year)
print(f'fv ={round(fv, 2): ,}')

fv = 1,032,300.53


In [42]:
def dr_rate(pv, fv, days, year=360):
    return (year/days) * (fv - pv)/fv

pv = 1_007_013
fv = 1_019_500
days = 120
year = 360

dr = dr_rate(pv, fv, days, year)
print(f'{dr = : 0.4f}')

dr =  0.0367


### Bond Equivalent Yield (BEY) or Investment Rate

The intent of *bond equivalent yield* (BEY) is to report to investors an interest rate for the security that is more meaningful than the discount rate and that allows a comparison to Treasury note and bond yields.

A **bond equivalent yield** (BEY) is a money market rate stated on a **365-day add-on rate basis**. 

$$BEY=\left(\frac{365}{Days}\right)\times\left(\frac{FV-PV}{PV}\right)$$
$$BEY=\left(\frac{365}{Days}\right)\times\left(\frac{\text{Dollar Discount}}{PV}\right)$$
$$BEY=\left(\frac{365}{Days}\right)\times\left(\frac{FV  \times DR \times \frac{Days}{Year}}{PV}\right)$$
$$BEY = \frac{365 \times DR}{Year_{\text{DR Convention}} - (Days \times DR)}$$


| Term    | Maturity Date | Discount Rate | Investment Rate | Price (per $100 in par value) |
| ------- | ------------- | ------------- | --------------- | ----------------------------- |
| 4 week  | 31/07/2008    | 1.850%         | 1.878%           | 99.856111                      |
| 13 week | 02/10/2008    | 1.900%         | 1.936%           | 99.519722                      |
| 26 week | 02/01/2009    | 2.135%         | 2.188%           | 98.914708                      |
| 52 week | 03/07/2009    | 2.295%         | 2.368%           | 97.679500                       |

In [101]:
settle = to_datetime("2008-7-3")
convention = 'actual/360'
mat_dr = {'2008-7-31': 0.0185,
             '2008-10-2': 0.019,
             '2009-1-2': 0.02135,
             '2009-7-3': 0.02295}

days_pv = {}
for mat, dr in mat_dr.items():
    days, year = days_year(settle, to_datetime(mat), convention=convention)
    price = dr_pv(100, dr, days, year)
    days_pv[days] = price
    bey = (365 * dr) / (360 - (days * dr)) # DR to BEY conversion formula
    print(f'{mat = } {price = :0.5f} {days = } {bey = :0.5f}')
    
print()

for days, price in days_pv.items():
    bey = addon_rate(price, 100, days, year=365) # AOR Cash flow formula set to 365 year
    print(f'{bey = :0.5f}')

mat = '2008-7-31' price = 99.85611 days = 28 bey = 0.01878
mat = '2008-10-2' price = 99.51972 days = 91 bey = 0.01936
mat = '2009-1-2' price = 98.91471 days = 183 bey = 0.02188
mat = '2009-7-3' price = 97.67312 days = 365 bey = 0.02382

bey = 0.01878
bey = 0.01936
bey = 0.02188
bey = 0.02382


### Comparing Discount Rate and Add-on Rate

General conversion formula between discount rates and addon rates (and vice versa) when quoted for the **same assumed number of days in the year**:

$$AOR = \frac{Year \times DR}{Year - (Days \times DR)}$$
$$DR = \frac{Year \times AOR}{Year + (Days \times AOR)}$$

Note that the DR, unlike an AOR, is **not an APR** because the $\frac{FV - PV}{FV}$ is not the periodic interest rate, it is the interest earned relative to the future value. This is not the way we think about an interest rate — the growth rate of an investment should be measured by the increase in value (FV − PV) given where we start (PV), not where we end (FV). The discount rate systematically understate the investor’s rate of return, as well as the borrower’s cost of funds. AOR will always be greater than the DR for the same cash flows, the more so the greater the number of days in the time period and the higher the level of interest rates.

In [24]:
def equiv_rate(rate, days, year, aor=True):
    if aor:
        return (year * rate) / (year - (days * rate))
    else:
        return (year * rate) / (year + (days * rate))
    

dr = 0.038
days = 180
year = 360

aor = equiv_rate(dr, days, year, aor=True)
print(f'{aor = }')

dr = equiv_rate(aor, days, year, aor=False)
print(f'{dr = }')

aor = 0.03873598369011213
dr = 0.038


### Converting Between Rate Periodicity 

In [81]:
quoted_apr = 0.12
from_freq = 12
to_freq = 1

print(bond_pricing.equiv_rate(rate=quoted_apr, from_freq=from_freq, to_freq=to_freq))

0.12682503013196977


### Examples

Suppose that a money market security can be purchased on January 12 for $64,000. The security matures on March 12, paying $65,000. Calculate the interest rate on the security to the nearest one-tenth of a basis point, given the following quotation methods and day-count conventions:
- Add-on Rate, Actual/360
- Add-on Rate, Actual/365
- Add-on Rate, 30/360
- Add-on Rate, Actual/370
- Discount Rate, Actual/360

In [53]:
pv = 64_000
fv = 65_000
settle = to_datetime('2025-1-12')
mat = to_datetime('2025-3-12')
convention = 'actual/360'

days, year = days_year(settle, mat, convention)
print(f'{days = }\n{year = }')
aor = addon_rate(pv, fv, days, year)
print(f'{aor = : 0.5f}')

days = 59
year = 360
aor =  0.09534


In [77]:
pv = 64_000
fv = 65_000
settle = to_datetime('2025-1-12')
mat = to_datetime('2025-3-12')
convention = 'actual/365'

days, year = days_year(settle, mat, convention)
print(f'{days = }\n{year = }')
aor = addon_rate(pv, fv, days, year)
print(f'{aor = : 0.5f}')

days = 59
year = 365
aor =  0.09666


In [48]:
pv = 64_000
fv = 65_000
settle = to_datetime('2025-1-12')
mat = to_datetime('2025-3-12')
convention = 'thirty/360'

days, year = days_year(settle, mat, convention)
print(f'{days = }\n{year = }')
aor = addon_rate(pv, fv, days, year)
print(f'{aor = : 0.5f}')

days = 60
year = 360
aor =  0.09375


In [50]:
pv = 64_000
fv = 65_000
settle = to_datetime('2025-1-12')
mat = to_datetime('2025-3-12')
convention = 'thirty/360'

days = days_year(settle, mat, convention)[0]
year = 370
print(f'{days = }\n{year = }')
aor = addon_rate(pv, fv, days, year)
print(f'{aor = : 0.5f}')

days = 60
year = 370
aor =  0.09635


In [51]:
pv = 64_000
fv = 65_000
settle = to_datetime('2025-1-12')
mat = to_datetime('2025-3-12')
convention = 'actual/360'

days, year = days_year(settle, mat, convention)
print(f'{days = }\n{year = }')
dr = dr_rate(pv, fv, days, year)
print(f'{dr = : 0.5f}')

days = 59
year = 360
dr =  0.09387
